# Introduction

Logistic Regression is binary classifier model that is used to classify a data point into either of two classes (0 or 1), popular examples include checking if a mail is spam or not etc. It is a generalized linear model i.e. it generates a linear model for this classification but uses some underlying method to account for the non-linearity in data. A logistic regression model looks something like this and it achieves this squiggly line using something called a sigmoid function.
<center><img src="images/logistic_regression_graph.png" style="width:340px;" ></center><br>

In this notebook I have explained how the logistic regression model achieves this line and has also compared my model to the popular library scikit-learns's logistic regression.

# Importing Libraries
Only importing the basic libraries to implement the logistic regression using maximum likelihood estimation and also the sklearn's logistic regression for the comparison using the accuaracy score

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Sigmoid Function
<center><img src="images/sigmoid-activation-function-1_0.png" style="width:340px;" ></center><br>
This is the sigmoid function whose domain is (-inf, inf) and range is [0, 1]. So we are going to find a linear equation that will be the input to our sigmoid function which in turn will give values ranging between 0 and 1 which will then be classified as 0 or 1 using a treshold value.

This is why logistic regression is called a Generalized Linear Model (GLM) even though its final line on the graph is not linear. Sigmoid functions graph is given below

<center><img src="images/sigmoid_graph.png" style="width:500px;" ></center><br>
Here I have defined the sigmoid function in Python

In [2]:
# Sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Log Likelihood Function
Log likelihood function can be defined as the log of product of probabilities of samples having target values 1 and 1 - probablities of samples having target values 0, This can be written mathematically as
<center><img src="images/probab_product.png" style="width:500px;" ></center><br>

Multiplying these terms together and taking their log gives us the log likeliood function. which on further simplyfing looks like this, where beta is the weights vector and x is the feature vector.
<center><img src="images/loglikelihood.png" style="width:500px;" ></center><br>

The Graph of log likelihood function looks something like this, the shape may change on the basis of parameters but the important point to note here is that it has a global maxima and no minimas.
<center><img src="images/log_likelihood_graph.png" style="width:500px;" ></center><br>

The implementation of log_likelihood function in Python is given below

In [3]:
# Log-likelihood function
def log_likelihood(X, y, beta):
    m = X.shape[0]
    ll = 0
    for i in range(m):
        xi = X[i]
        yi = y[i]
        z = np.dot(beta, xi)
        ll += yi * z - np.log(1 + np.exp(z))
    return ll

# Maximum Likelihood Estimation
The log likelihood function defined above is the measure of how good a model is, the greater the log likelihood function is the better the model is, So our aim is to maximise the log likelihood function or obtain those weights for which the log likelihood is maximum.

There are various methods to maximise the log likelihood function but in this notebook I will be using the Newton-Raphson method.

# Newton Raphson on Log Likelihood
The newton raphson method is used to find the solution of an equation by performing iterations and in each iteration going closer to the solution. We know the log likelihood function has only a global maxima so the roots of derivative of the log likelihood function will give us the value of parameters for which it will be maximum.
<center><img src="images/newton_raphson.png" style="width:500px;" ></center><br>

where Xn+1 is our new value of parameter, Xn is the current value of parameter, f(Xn) will be current value of gradient (first derivative) of log likelihood i.e. the equation to be solved and f'(Xn) will be the hessian (second derivative) of log likelihood.

I have impolemented the functions to perform this in the following cells: 

In [4]:
# First derivative of log-likelihood with respect to beta
def log_likelihood_gradient(X, y, beta):
    m = X.shape[0]
    gradient = np.zeros_like(beta)
    for i in range(m):
        xi = X[i]
        yi = y[i]
        pi = sigmoid(np.dot(beta, xi))
        gradient += xi * (yi - pi)
    return gradient

In [5]:
# Second derivative (Hessian) of log-likelihood with respect to beta
def log_likelihood_hessian(X, beta):
    m = X.shape[0]
    n = X.shape[1]
    hessian = np.zeros((n, n))
    for i in range(m):
        xi = X[i]
        pi = sigmoid(np.dot(beta, xi))
        hessian += pi * (1 - pi) * np.outer(xi, xi)
    return hessian

In [6]:
# Newton-Raphson method to estimate beta
def newton_raphson(X, y, beta, max_iterations=100, tolerance=1e-6):
    iteration = 0
    while iteration < max_iterations:
        gradient = log_likelihood_gradient(X, y, beta)
        hessian = log_likelihood_hessian(X, beta)
        beta_new = beta + np.linalg.inv(hessian).dot(gradient)
        if np.all(np.abs(beta_new - beta) < tolerance):
            break
        beta = beta_new
        iteration += 1
    return beta

# Training My implementation of logistic regression
Here I have used a very small random data for logistic regression, I imported this data then i split it into test and train data and used the train data to train my self implementation of logistic regression, I have also added the bias term i.e. a constant that is added to our linear equation obtained to get more accurate prediction.

Thus or final model looks something like this for the data I used that has two features named "x1 and "x2".
<center><img src="images/final_model.jpeg" style="width:500px;" ></center><br>

where b is our bias term and a1 and a2 are our weights for feature x1 and x2 respectively.
the p(x) thus obtained is then classified as either 1 or 0 based on if it is greater than a treshold value or not. Here I have used 0.5 as treshold value.

In [7]:
# Load your dataset
data = pd.read_csv("data/data.csv")

# Assume the last column is the target variable y, and the rest are features X
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Add intercept term to X
X_train_final = np.c_[np.ones(X_train.shape[0]), X_train]
X_test_final = np.c_[np.ones(X_test.shape[0]), X_test]

# Initialize beta (including intercept)
beta_initial = np.zeros(X_train_final.shape[1])

# Apply Newton-Raphson to find optimal beta
beta_hat = newton_raphson(X_train_final, y_train, beta_initial)

In [9]:
# Print the estimated coefficients
print("Estimated Coefficients:", beta_hat)

# Predict on test set
y_pred_probs = sigmoid(np.dot(X_test_final, beta_hat))
y_pred = (y_pred_probs >= 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Estimated Coefficients: [-30.48713132   0.26212704   0.22697362]
Accuracy: 0.8


# Comparison with Scikit Learn's logistic regression
We can see for this small data my logistic regression and scikit learn's logistic regression both perform almost the same

In [10]:
# Initialize logistic regression model
model = LogisticRegression(random_state=42)

# Train the model
model.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [11]:
# Make predictions on the test set
y_pred_2 = model.predict(X_test)

coefficients = model.coef_
intercept = model.intercept_

print("EstimatedCoefficients:", intercept, coefficients[0])

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_2)
print(f"Accuracy: {accuracy}")

EstimatedCoefficients: [-30.20000099] [0.25970232 0.22481016]
Accuracy: 0.8
